In [61]:
import cv2, imp, os
from PIL import Image, ImageDraw
import numpy as np

In [51]:
print(cv2.__version__)

jpg_root = './../../lc_files/20181218_jpg/1846IMP001006318/page_4.jpg'


img = cv2.imread(jpg_root, 0)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# img = cv2.bitwise_not(img)
kernel = np.ones((2,2),np.uint8)
erosion = cv2.erode(img,kernel,iterations = 1)
image = Image.fromarray(erosion)
# image.show()
saved_path = "./cvtest.jpg"
image.save(saved_path, "JPEG")

3.3.0-dev
(2338, 1653)


In [77]:
import package.utils as utils
import package.visionapi as visionapi
import package.augmentation as imgaug
import service

imp.reload(utils)
imp.reload(visionapi)
imp.reload(imgaug)
imp.reload(service)

jpg_root = './../../lc_files/20181218_jpg/1846IMP001006318'
jpg_list = ['./../../lc_files/20181218_jpg/1846IMP001006318/page_1.jpg', 
            './../../lc_files/20181218_jpg/1846IMP001006318/page_2.jpg', 
            './../../lc_files/20181218_jpg/1846IMP001006318/page_3.jpg', 
            './../../lc_files/20181218_jpg/1846IMP001006318/page_4.jpg', 
            './../../lc_files/20181218_jpg/1846IMP001006318/page_5.jpg', 
            './../../lc_files/20181218_jpg/1846IMP001006318/page_6.jpg']

credential = '../../service_acc.json'
visionapi.set_vision_credential(credential)
# res = visionapi.annotateDocument(saved_path)
service.annotateCreditLetter(credential, '0001', jpg_list, jpg_root)

['./../../lc_files/20181218_jpg/1846IMP001006318/page_1_aug.jpg', './../../lc_files/20181218_jpg/1846IMP001006318/page_2_aug.jpg', './../../lc_files/20181218_jpg/1846IMP001006318/page_3_aug.jpg', './../../lc_files/20181218_jpg/1846IMP001006318/page_4_aug.jpg', './../../lc_files/20181218_jpg/1846IMP001006318/page_5_aug.jpg', './../../lc_files/20181218_jpg/1846IMP001006318/page_6_aug.jpg']


{'checklist': {'additional_docs': {'original_47A': '1. INVOICES IN EXCESS OF DRAFT AMOUNT ARE NOT ACCEPTABLE.\n2. THIRD PARTY BILL OF LADING ARE NOT ACCEPTABLE.\n3. ANY DOCUMENTS CALLED FOR HEREIN PRODUCED BY REPROGRAPHIC\nPROCESS OR COMPUTERISED SYSTEM ARE ACCEPTABLE, PROVIDED ALL SUCH\nORIGINAL DOCUMENTS ARE MANUALLY SIGNED.\n4. INVOICE, PACKING LIST AND BILL OF LADING MUST BE MARKED THESE\nGOODS ARE BEING IMPORTED UNDER OGL AND IEC NO.0694005258.\n5. ALL THE DOCUMENTS MUST BE DRAWN IN ENGLISH LANGUAGE.\n6. BILL OF EXCHANGE, BILL OF LADING, INVOICE DATED PRIOR TO THIS\nCREDIT ARE NOT ACCEPTABLE..\n7. ALL DOCUMENTS EXCEPT CERTIFICATE OF ORIGIN TO BE MARKED DRAWN\nUNDER DOCUMENTARY CREDIT NO. 1846IMPO01006318 AND DATE 22/06/2018\x08MALL,\nUTTAR PRADESH, INDIA\n8. BILL OF LADING MUST BE DATED NOT LATER THAN 20/08/2018 NOR\nPRIOR TO THE DATE OF THIS CREDIT. BILL OF EXCHANGE MUST BE DATED\nAND PRESENTED OR NEGOTIATED NOT LATER THAN 10 /2018 AT THE\nCOUNTERS OF ADVISING BANK OR NEGOTIATING

In [65]:
tmp = os.path.basename(jpg_root)
print(tmp)

page_4.jpg
